In [11]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.base import ClusterMixin
from sklearn.base import BaseEstimator
sns.set(rc={'figure.figsize':(11.7,8.27)})
from random import *
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import dendrogram
from sklearn.manifold import MDS
from scipy.spatial.distance import cdist

In [12]:
runner = pd.read_csv("../../../output/data_clean/cleaned_runner.csv", index_col=0)
race = pd.read_csv("../../../output/data_clean/cleaned_race_2.csv", index_col= 0)
runner.age_range = pd.Categorical(
    runner.age_range, categories=['16-25','26-40','40-60','61+','Undefined'],ordered=True
)
runner.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 130137 entries, 0 to 137802
Data columns (total 13 columns):
 #   Column            Non-Null Count   Dtype   
---  ------            --------------   -----   
 0   race_year_id      130137 non-null  int64   
 1   ranking           113523 non-null  float64 
 2   runner            130137 non-null  object  
 3   time              113523 non-null  object  
 4   age               128764 non-null  float64 
 5   gender            130108 non-null  object  
 6   nationality       130137 non-null  object  
 7   time_in_seconds   113523 non-null  float64 
 8   runner_id         130108 non-null  object  
 9   age_course        128764 non-null  float64 
 10  age_range         130137 non-null  category
 11  speed             113523 non-null  float64 
 12  equivalent_speed  113523 non-null  float64 
dtypes: category(1), float64(6), int64(1), object(5)
memory usage: 13.0+ MB


In [13]:
one_hot_gender = pd.get_dummies(runner['gender'], prefix = "gender")
runner = runner.drop('gender',axis = 1)
runner = runner.join(one_hot_gender)
runner = runner.dropna()

In [14]:
join = pd.merge(runner, race, left_on="race_year_id", right_on="race_year_id", how="left")
join = join.loc[join.speed < 20]
join = join.loc[join.distance > 150]

In [15]:
runner.ranking = join.ranking / join.frequency_in_ranking *100

In [16]:
runner.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 112186 entries, 0 to 137802
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype   
---  ------            --------------   -----   
 0   race_year_id      112186 non-null  int64   
 1   ranking           87857 non-null   float64 
 2   runner            112186 non-null  object  
 3   time              112186 non-null  object  
 4   age               112186 non-null  float64 
 5   nationality       112186 non-null  object  
 6   time_in_seconds   112186 non-null  float64 
 7   runner_id         112186 non-null  object  
 8   age_course        112186 non-null  float64 
 9   age_range         112186 non-null  category
 10  speed             112186 non-null  float64 
 11  equivalent_speed  112186 non-null  float64 
 12  gender_M          112186 non-null  uint8   
 13  gender_W          112186 non-null  uint8   
dtypes: category(1), float64(6), int64(1), object(4), uint8(2)
memory usage: 10.6+ MB


In [17]:
def get_new_runners_per_year(df):
    years = df.year.unique().tolist()[::-1][1:]
    new_runners = []
    
    for year in years:
        search_df = df.loc[df.year < year]
        current_year_df = df.loc[df.year == year]
        unique_runners = current_year_df.runner_id.unique()
        new_year_runner = 0
        
        for runner in unique_runners:
            print(runner)
            if search_df.loc[search_df.runner_id == runner]:
                new_year_runner += 1
        new_runners.append(new_year_runner)
            
    return years

In [18]:
get_new_runners_per_year(join)

CATMUREdwardGBRM39.0


ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().